## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [ ]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, use_current_place_point_only, use_current_sub_binders_only, finish_L_shape, set_l_shape_object, ModeSwitcher

## init combined robot config

In [ ]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

In [ ]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

## create scene builder

In [ ]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
# deprecated: s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [ ]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# save_pickle("xyz_rpy_robots.pkl", xyz_rpy_robots)
xyz_rpy_robots = load_pickle("xyz_rpy_robots.pkl")
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

## init planning pipeline

In [ ]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [ ]:
# gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

In [ ]:
self=s_builder
item_names=None
level_mask=[DetectionLevel.ENVIRONMENT]
color=(0.6,0.6,0.6,1)
collision=True
gscene=gscene
visualize=False
    
# xyz_rpy_dict = self.detect_items(item_names=item_names, level_mask=level_mask, visualize=visualize)
# save_pickle("env_dict.pkl", xyz_rpy_dict)
xyz_rpy_dict = load_pickle("env_dict.pkl")
gtem_dict = {}
for ename, xyzrpy in xyz_rpy_dict.items():
    kwargs = dict(name=ename, center=xyzrpy[0], rpy=xyzrpy[1], color=color,
                  link_name=self.base_link, collision=collision)
    kwargs.update(self.detector.get_geometry_kwargs(ename))
    gtem_dict[ename] = gscene.create_safe(**kwargs)

In [ ]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

## add indy tool

In [ ]:
add_indy_sweep_tool(gscene, "indy0", face_name="brush_face")

## add sweep face

In [ ]:
# track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]

In [ ]:
track = gtem_dict["track"]

In [ ]:
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [ ]:
gscene.update_markers_all()

## add wp

In [ ]:
TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.17
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,TRC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(1e-2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))

In [ ]:
gscene.update_markers_all()

## add blocking structure on table

In [ ]:
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

## add box

In [ ]:
# gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [ ]:
self=s_builder
item_names=None
level_mask=[DetectionLevel.MOVABLE]
color=(0.6,0.6,0.6,1)
collision=True
gscene=gscene
visualize=False
    
# xyz_rpy_dict = self.detect_items(item_names=item_names, level_mask=level_mask, visualize=visualize)
# save_pickle("obj_dict.pkl", xyz_rpy_dict)
xyz_rpy_dict = load_pickle("obj_dict.pkl")
gtem_dict = {}
for ename, xyzrpy in xyz_rpy_dict.items():
    kwargs = dict(name=ename, center=xyzrpy[0], rpy=xyzrpy[1], color=color,
                  link_name=self.base_link, collision=collision)
    kwargs.update(self.detector.get_geometry_kwargs(ename))
    gtem_dict[ename] = gscene.create_safe(**kwargs)

In [ ]:
gscene.update_markers_all()

## Register binders

In [ ]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [ ]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), 
                     rpy=(0,np.pi,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)

## add objects

In [ ]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [ ]:
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [ ]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [ ]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)

### planners

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [ ]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
# checkers_all = [rcheck, gcheck]
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [ ]:
mplan.motion_filters = checkers_all

In [ ]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [ ]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

In [ ]:
pscene.subject_name_list

# Node Sampler

In [ ]:
# from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

# tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
# tplan.parent_node_sampler = UniformNodeSampler(3)
# # tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [ ]:
# from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
# tplan.custom_rule = SweepEntranceControlRule(pscene)

In [ ]:
# from copy import deepcopy
# from ....utils.utils import *
# from ...constraint.constraint_subject import AbstractObject, SweepLineTask, SubjectType
from pkg.planning.task.custom_rules.interface import CustomRuleInterface


##
# @class SweepEntranceControlRule
# @brief Strictly control entrance to sweep tasks, as they are fully defined and no need to be sampled multiple times
class SweepEntranceControlRule(CustomRuleInterface):
    def __init__(self, pscene):
        self.pscene = pscene
        self.chain_dict = pscene.robot_chain_dict
        self.NUM_WP_TRIALS = 1
        self.refoliate_flag = True

    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager

        no_enter_sidxes = [stype == SweepLineTask for stype in enumerate(self.pscene.subject_type_list)]
        no_enter_initials = [tplan.initial_state.node[sidx] for sidx in no_enter_sidxes]
        self.sorted_unstop_keys = sorted(tplan.unstoppable_terminals.keys())

        self.node_dict = {}
        self.enter_dict = {}
        for node, leafs in tplan.node_dict.items():
            self.node_dict[node] = deepcopy(leafs)
            if not all([node[k] in terms for k, terms in tplan.unstoppable_terminals.items()]):
                tplan.node_dict[node] = set([])  ## unstoppable node change will be reserved by this custom rule
            else:
                # entering to unstoppable terminal is controlled by this rule
                self.enter_dict[node] = set([leaf for leaf in leafs
                                             if any(
                        [leaf[k] not in terms for k, terms in tplan.unstoppable_terminals.items()])
                                             and all(
                        [leaf[k] >= leaf[k + 1] for k in self.sorted_unstop_keys[:-1]])])  # the task is done in order
                tplan.node_dict[node] = set([leaf for leaf in leafs
                                             if
                                             all([leaf[k] in terms for k, terms in tplan.unstoppable_terminals.items()])
                                             ])

        self.node_parent_dict = {}
        for node, parents in tplan.node_parent_dict.items():
            self.node_parent_dict[node] = deepcopy(parents)
            tplan.node_parent_dict[node] = set(
                [parent for parent in parents  ## unstoppable node change will be reserved by this custom rule
                 if all([parent[k] in terms for k, terms in tplan.unstoppable_terminals.items()])])

    ##
    # @brief    re-foliate for some defined cases
    # @remark   case 1: meaningless homing - stayed on same node for two turns and do action of same kind \n
    #           case 2: already-moved object
    def refoliate(self, tplan, new_node, parent_sidx):
        if not self.refoliate_flag:
            return new_node, parent_sidx, False
        else:
            reject = False
            parent_snode = tplan.snode_dict[parent_sidx]
            parent_node = parent_snode.state.node
            anc_nodes = [tplan.snode_dict[pidx].state.node for pidx in parent_snode.parents]
            if new_node != parent_node:  # this is not homing motion
                # subject_gname_list = [obj.geometry.name for obj in self.pscene.subject_dict.values()
                #                       if obj.stype == SubjectType.OBJECT]
                active_binder_geo = [b for a, b in zip(parent_node, new_node) if a != b][0]
                if any([active_binder_geo in subject.geometry.get_family()
                        for subject in self.pscene.subject_dict.values() if isinstance(subject, AbstractObject)]):     # don't put on other object
                    reject = True
                    return new_node, parent_sidx, reject

                if len(anc_nodes) > 1 and anc_nodes[-1] == parent_node:  # previous motion was homing
                    if ([a for a, b in zip(anc_nodes[-2], parent_node) if a != b][0]
                            == [b for a, b in zip(parent_node, new_node) if a != b][0]):  # use same binder after homing
                        parent_sidx = parent_snode.parents[-1]
                        parent_snode = tplan.snode_dict[parent_sidx]
                        parent_node = parent_snode.state.node
                        anc_nodes = anc_nodes[:-1]
                if len(anc_nodes) > 0 and new_node in anc_nodes:  # already-moved object
                    active_binder_geo = [b for a, b in zip(parent_node, new_node) if a != b][0]
                    if ((active_binder_geo not in self.pscene.geometry_actor_dict)  # active binder is doing Task
                            or
                            (self.pscene.actor_robot_dict[self.pscene.geometry_actor_dict[active_binder_geo][0]]
                             is not None)  # active binder is controllable actor
                    ):
                        first_access_idx = anc_nodes.index(new_node)
                        if first_access_idx == 0:  # this is returning to initial state
                            reject = True
                        else:  # foliate from before first move of this object
                            parent_sidx = parent_snode.parents[first_access_idx - 1]
                            parent_snode = tplan.snode_dict[parent_sidx]
                            parent_node = parent_snode.state.node
                            anc_nodes = anc_nodes[:first_access_idx - 1]
            return new_node, parent_sidx, reject

    def __call__(self, tplan, snode_src, snode_new, connection_result):
        #         print("CustomRule call")
        stack_res = False
        stack_items = []
        if connection_result and snode_new.state.node in self.enter_dict:
            with tplan.snode_dict_lock:
                nb_nodes = list(tplan.neighbor_nodes.keys())
                for nb_node in nb_nodes:
                    for ukey in self.sorted_unstop_keys:
                        if nb_node[ukey] < snode_new.state.node[ukey]:
                            del tplan.neighbor_nodes[nb_node]
                            break  # task is done in order

        if connection_result and snode_new.state.node in self.enter_dict:
            stack_res, stack_items = True, list(self.enter_dict[snode_new.state.node])

        if snode_src is not None:

            node_src = snode_src.state.node
            node_new = snode_new.state.node
            diff_sidxes = np.where([ntem_s != ntem_g for ntem_s, ntem_g in zip(node_src, node_new)])[0]
            #             print("{}->{} , diff: {}".format(node_src, node_new, diff_sidxes))
            if len(diff_sidxes) == 0:
                return stack_res, stack_items
            diff_sidx = diff_sidxes[0]
            diff_sname = self.pscene.subject_name_list[diff_sidx]
            diff_subject = self.pscene.subject_dict[diff_sname]

            if isinstance(diff_subject, SweepLineTask): # Sweep entrance control rule
                #                 print("Rule for SweepLineTask")
                with tplan.snode_dict_lock:
                    if node_src[diff_sidx] not in tplan.unstoppable_terminals[diff_sidx]:
                        # from intermediate wp -> remove access to them
                        snode_list = tplan.node_snode_dict[node_src]
                        if snode_src.idx in snode_list:
                            # print("Removing {} from {}: {} not in {}".format(
                            #     snode_src.idx, snode_list,
                            #     node_src[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            # print(snode_list)
                            snode_list.remove(snode_src.idx)
                            tplan.node_snode_dict[node_src] = snode_list
                            print(tplan.node_snode_dict[node_src])
                        if len(snode_list) == 0 and node_new in tplan.neighbor_nodes:
                            del tplan.neighbor_nodes[node_new]
                    if connection_result:
                        if node_new[diff_sidx] in tplan.unstoppable_terminals[diff_sidx]:  ## in terminal condition
                            # print("Check home: {} in {}".format(
                            #     node_new[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            link_name = self.pscene.gscene.NAME_DICT[
                                snode_new.state.binding_state[diff_sidx][-1]].link_name
                            rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if
                                            link_name in chain_vals['link_names']]
                            if len(rname_candis) == 0:
                                print("no robot candis")
                            else:
                                #                                 print("reserve homing = {}".format(snode_new.idx))
                                newstate = snode_new.state.copy(self.pscene)
                                jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                                newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                                return True, [newstate] + stack_items
                        else:  ## not in terminal condition
                            # print("Reserve next wp from {}".format(snode_new.idx))
                            snode_list = tplan.node_snode_dict[node_new]
                            if snode_new.idx in snode_list:
                                # print("Removing {} from {} as next wp is reserved".format(snode_new.idx, snode_list))
                                snode_list.remove(snode_new.idx)
                                tplan.node_snode_dict[node_new] = snode_list
                            next_node_candis = list(self.node_dict[node_new])
                            assert len(
                                next_node_candis) == 1, "non-terminal sweep task should have only 1 leaf ({}) {}-{}".format(
                                diff_sidx, node_new, next_node_candis)
                            return True, next_node_candis * self.NUM_WP_TRIALS + stack_items
                return stack_res, stack_items

            if isinstance(diff_subject, AbstractObject):
                #                 print("Rule for AbstractObject")
                if connection_result:
                    link_name1 = snode_src.state.state_param[diff_sname][0]
                    link_name2 = snode_new.state.state_param[diff_sname][0]
                    rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if
                                    link_name1 in chain_vals['link_names'] or link_name2 in chain_vals['link_names']]
                    if len(rname_candis) == 0:
                        print("no robot candis")
                        return stack_res, stack_items
                    else:
                        #                         print("try homing")
                        newstate = snode_new.state.copy(self.pscene)
                        jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                        newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                        return True, [newstate] + stack_items
        return stack_res, stack_items

In [ ]:
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan & show

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset()

In [ ]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
#     gscene.clear_non_fixed()
    
#     ## detect again
#     gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        with gtimer.block(gname):
            box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
            box_new.register_binders(pscene, PlacePlane)
            box_list.append(box_new)
            
initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
gscene.update_markers_all()

# remove place points and sub-binders except for the current ones
use_current_place_point_only(pscene, initial_state)
use_current_sub_binders_only(pscene, initial_state)
tplan.prepare()
mplan.update_gscene()

In [ ]:
print(gtimer)

## w/o redundant motions

In [ ]:
mplan.reset_log(False)
gtimer.reset()
tplan.custom_rule.refoliate_flag = True

# with gtimer.block("detect"):
#     for sname in pscene.subject_name_list:
#         if isinstance(pscene.subject_dict[sname], AbstractObject):
#             pscene.remove_subject(sname)
            
#     gscene.clear_non_fixed()
    
#     ## detect again
#     gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

#     ## create boxes
#     box_list =  [] 
#     for gname in sorted(gtem_dict.keys()):
#         box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
#         box_new.register_binders(pscene, PlacePlane)
#         box_list.append(box_new)
    
#     initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
#     gscene.update_markers_all()

#     # remove place points and sub-binders except for the current ones
#     use_current_place_point_only(pscene, initial_state)
#     use_current_sub_binders_only(pscene, initial_state)
#     tplan.prepare()
#     mplan.update_gscene()
    
print(initial_state.node)

result_list = []
traj_list = []
trans_list = []
snode_schedule_all_stack = []
for _ in range(10):
    with gtimer.block("one_plan", stack=True):
        gtimer.tic("firstmove")
        obj_num = len(gtem_dict)
        sweep_num = len(sweep_list)
        from_state = initial_state
        t_exe = None
        snode_schedule_all = []
        traj_len = 0
        for sweep_idx in range(sweep_num):
            gcheck.put_banned = [track_list[sweep_idx][2]]
            sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
        #     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
            goal_nodes = [("track_face",)*obj_num+sweep_goal]
            if sweep_idx < sweep_num-1:
                for i_s in range(obj_num):
                    obj_goal = ["track_face"]*obj_num
                    obj_goal[i_s] = "grip1"
                    goal_nodes += [tuple(obj_goal)+sweep_goal]
            gtimer.tic("plan{}".format(sweep_idx))
            ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                          timeout_loop=200, multiprocess=True, timeout=10,
                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
            gtimer.toc("plan{}".format(sweep_idx), stack=True)
            schedules = ppline.tplan.find_schedules()
            if len(schedules)==0:
                break
            schedules_sorted = ppline.tplan.sort_schedule(schedules)
            snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
            snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
            snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
#             snode_schedule = snode_schedule_safe
            snode_schedule = mix_schedule(mplan, snode_schedule_safe)
            from_state = snode_schedule[-1].state
            if t_exe:
                t_exe.join()
            else:
                if len(snode_schedule_all)==0:
                    gtimer.toc("firstmove")
            snode_schedule_all.append(snode_schedule)
        result_list.append(len(schedules)!=0)
        if len(schedules)!=0:
            traj_list.append([snode_schedule[-1].traj_tot for snode_schedule in snode_schedule_all])
            trans_list.append([len(snode_schedule[-1].parents) for snode_schedule in snode_schedule_all])
            snode_schedule_all_stack.append(snode_schedule_all)
        #     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
        #     t_exe.start()
        # t_exe.join()

In [ ]:
print("{} % : {} / {} / {} / {}".format(np.mean(result_list)*100,
    np.min(np.sum(traj_list, axis=-1)), np.mean(np.sum(traj_list, axis=-1)), 
    np.median(np.sum(traj_list, axis=-1)), np.max(np.sum(traj_list, axis=-1))))

In [ ]:
print("{} % : {} / {} / {} / {}".format(np.mean(result_list)*100,
    np.min(np.sum(trans_list, axis=-1)), np.mean(np.sum(trans_list, axis=-1)), 
    np.median(np.sum(trans_list, axis=-1)), np.max(np.sum(trans_list, axis=-1))))

In [ ]:
snode_schedule_all_stack_1 = snode_schedule_all_stack

In [ ]:
print("total : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000))

print("plan0 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000))
print("plan1 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000))
print("plan2 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000))

In [ ]:
print(gtimer)

In [ ]:
play_schedule_clearance_highlight_full(ppline, snode_schedule_all_stack_1[0], tcheck)

In [ ]:
self = ppline
snode_schedule = snode_schedule_all_stack_1[0][0]
period = 0.001
tcheck = tcheck

In [ ]:
play_schedule_clearance_highlight(ppline, snode_schedule, tcheck=tcheck, period=0.001)

## w. redundant motion

In [ ]:
mplan.reset_log(False)
gtimer.reset()
tplan.custom_rule.refoliate_flag = False

# with gtimer.block("detect"):
#     for sname in pscene.subject_name_list:
#         if isinstance(pscene.subject_dict[sname], AbstractObject):
#             pscene.remove_subject(sname)
            
#     gscene.clear_non_fixed()
    
#     ## detect again
#     gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

#     ## create boxes
#     box_list =  [] 
#     for gname in sorted(gtem_dict.keys()):
#         box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
#         box_new.register_binders(pscene, PlacePlane)
#         box_list.append(box_new)
    
#     initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
#     gscene.update_markers_all()

#     # remove place points and sub-binders except for the current ones
#     use_current_place_point_only(pscene, initial_state)
#     use_current_sub_binders_only(pscene, initial_state)
#     tplan.prepare()
#     mplan.update_gscene()
    
print(initial_state.node)

result_list = []
traj_list = []
trans_list = []
snode_schedule_all_stack = []
for _ in range(10):
    with gtimer.block("one_plan", stack=True):
        gtimer.tic("firstmove")
        obj_num = len(gtem_dict)
        sweep_num = len(sweep_list)
        from_state = initial_state
        t_exe = None
        snode_schedule_all = []
        traj_len = 0
        for sweep_idx in range(sweep_num):
            gcheck.put_banned = [track_list[sweep_idx][2]]
            sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
        #     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
            goal_nodes = [("track_face",)*obj_num+sweep_goal]
            if sweep_idx < sweep_num-1:
                for i_s in range(obj_num):
                    obj_goal = ["track_face"]*obj_num
                    obj_goal[i_s] = "grip1"
                    goal_nodes += [tuple(obj_goal)+sweep_goal]
            gtimer.tic("plan{}".format(sweep_idx))
            ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                          timeout_loop=200, multiprocess=True, timeout=10,
                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
            gtimer.toc("plan{}".format(sweep_idx), stack=True)
            schedules = ppline.tplan.find_schedules()
            if len(schedules)==0:
                break
            schedules_sorted = ppline.tplan.sort_schedule(schedules)
            snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
            snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
            snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
#             snode_schedule = snode_schedule_safe
            snode_schedule = mix_schedule(mplan, snode_schedule_safe)
            from_state = snode_schedule[-1].state
            if t_exe:
                t_exe.join()
            else:
                if len(snode_schedule_all)==0:
                    gtimer.toc("firstmove")
            snode_schedule_all.append(snode_schedule)
        result_list.append(len(schedules)!=0)
        if len(schedules)!=0:
            traj_list.append([snode_schedule[-1].traj_tot for snode_schedule in snode_schedule_all])
            trans_list.append([len(snode_schedule[-1].parents) for snode_schedule in snode_schedule_all])
            snode_schedule_all_stack.append(snode_schedule_all)
        #     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
        #     t_exe.start()
        # t_exe.join()

In [ ]:
print("{} % : {} / {} / {} / {}".format(np.mean(result_list)*100,
    np.min(np.sum(traj_list, axis=-1)), np.mean(np.sum(traj_list, axis=-1)), 
    np.median(np.sum(traj_list, axis=-1)), np.max(np.sum(traj_list, axis=-1))))

In [ ]:
print("{} % : {} / {} / {} / {}".format(np.mean(result_list)*100,
    np.min(np.sum(trans_list, axis=-1)), np.mean(np.sum(trans_list, axis=-1)), 
    np.median(np.sum(trans_list, axis=-1)), np.max(np.sum(trans_list, axis=-1))))

In [ ]:
snode_schedule_all_stack_2 = snode_schedule_all_stack

In [ ]:
print("total : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000))

print("plan0 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000))
print("plan1 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000))
print("plan2 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000))

In [ ]:
for snode_schedule_all in snode_schedule_all_stack_1:
    for snode_schedule in snode_schedule_all:
        ppline.play_schedule(snode_schedule, period=0.001)

In [ ]:
ppline.play_schedule(snode_schedule_all_stack_2[2][0], period=0.001)

In [ ]:
ppline.play_schedule(snode_schedule_all_stack_2[2][2], period=0.001)

In [ ]:
print(gtimer)

In [ ]:
gtimer.timelist_dict['one_plan']

In [ ]:
print("{} % : {} / {} / {} / {}".format(np.mean(result_list),
                                        np.min(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000))

## rule + Projected

In [ ]:
print(gtimer)

In [ ]:
gtimer.timelist_dict['one_plan']

In [ ]:
np.mean(result_list)

In [ ]:
np.min(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.mean(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.median(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.max(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

## no rule

In [ ]:
np.mean(result_list)

In [ ]:
np.min(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.mean(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.median(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.max(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
print(gtimer)

In [ ]:
gtimer.timelist_dict['one_plan']

In [ ]:
result_list

In [ ]:
np.min(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.mean(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.median(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

In [ ]:
np.max(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000

## mix full schedule

In [ ]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

with gtimer.block("mix_schedule_safe"):
    safe_mixed = mix_schedule(mplan, snode_schedule_cat)

print(gtimer)

### play schedule

In [ ]:
ppline.play_schedule(safe_mixed, period=0.001)

### Run fully mixed schedule

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,False)
time.sleep(1)
crob.grasp(True, True)
time.sleep(2)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
ppline.execute_schedule(safe_mixed, mode_switcher=ModeSwitcher(pscene))

## Plan & execute

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(False,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(1)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)

    gscene.clear_non_fixed()

    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)

    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()

print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=100, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        gtimer.toc("firstmove")
    t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), kwargs=dict(auto_stop=False, mode_switcher=ModeSwitcher(pscene)))
    t_exe.start()
    snode_schedule_all.append(snode_schedule)
t_exe.join()
crob.stop_tracking()

In [ ]:
print(gtimer)

In [ ]:
for _ in range(3):
    indy = crob.robot_dict['indy0']
    with indy:
        indy.wait_di(16)

    mplan.reset_log(False)
    gtimer.reset()

    with gtimer.block("detect"):
        for sname in pscene.subject_name_list:
            if isinstance(pscene.subject_dict[sname], AbstractObject):
                pscene.remove_subject(sname)
            
        gscene.clear_non_fixed()

        ## detect again
        gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
    
        ## create boxes
        box_list =  [] 
        for gname in sorted(gtem_dict.keys()):
            box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
            box_new.register_binders(pscene, PlacePlane)
            box_list.append(box_new)

        initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
        gscene.update_markers_all()

        # remove place points and sub-binders except for the current ones
        use_current_place_point_only(pscene, initial_state)
        use_current_sub_binders_only(pscene, initial_state)
        tplan.prepare()
        mplan.update_gscene()

    print(initial_state.node)

    gtimer.tic("firstmove")
    obj_num = len(gtem_dict)
    sweep_num = len(sweep_list)
    from_state = initial_state
    t_exe = None
    snode_schedule_all = []
    for sweep_idx in range(sweep_num):
        gcheck.put_banned = [track_list[sweep_idx][2]]
        sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
    #     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
        goal_nodes = [("track_face",)*obj_num+sweep_goal]
        if sweep_idx < sweep_num-1:
            for i_s in range(obj_num):
                obj_goal = ["track_face"]*obj_num
                obj_goal[i_s] = "grip1"
                goal_nodes += [tuple(obj_goal)+sweep_goal]
        gtimer.tic("plan{}".format(sweep_idx))
        ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                      timeout_loop=20, multiprocess=True, timeout=5,
                      plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
        gtimer.toc("plan{}".format(sweep_idx))
        schedules = ppline.tplan.find_schedules()
        schedules_sorted = ppline.tplan.sort_schedule(schedules)
        snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
        snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
        snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
        snode_schedule = mix_schedule(mplan, snode_schedule_safe)
        from_state = snode_schedule[-1].state
        if t_exe:
            t_exe.join()
        else:
            gtimer.toc("firstmove")
        t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), kwargs=dict(auto_stop=False, mode_switcher=ModeSwitcher(pscene)))
        t_exe.start()
        snode_schedule_all.append(snode_schedule)
    t_exe.join()
    crob.stop_tracking()

In [ ]:
print(gtimer)

In [ ]:
traj_dur_list_list = []
for snode_schedule_all in snode_schedule_all_stack:
    traj_dur_list = []
    for snode_schedule in snode_schedule_all:
        traj_dur = 0
        for snode in snode_schedule:
            if snode.traj is not None:
                traj_dur += len(snode.traj)
        traj_dur_list.append(traj_dur)
    traj_dur_list_list.append(traj_dur_list)

In [ ]:
traj_freq = 50

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
 
plan0T=np.mean(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000
plan1T=np.mean(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000
plan2T=np.mean(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000
# Rainfall data - 2nd day of the month for plotting broken horizontal bar chart

yrange = (5,1)
xranges = [(0,plan0T)]
plt.broken_barh(xranges, yrange, facecolors='red')
yrange = (4,1)
xranges = [(plan0T,plan1T)]
plt.broken_barh(xranges, yrange, facecolors='red')
yrange = (3,1)
xranges = [(plan0T+plan1T,plan2T)]
plt.broken_barh(xranges, yrange, facecolors='red')


plan_full = plan0T+plan1T+plan2T
exe0, exe1, exe2 = np.mean(np.divide(np.array(traj_dur_list_list, dtype=np.float), traj_freq), axis=0)

xranges = [(plan_full,exe0)]
yrange = (2,1)
plt.broken_barh(xranges, yrange, facecolors='blue')

xranges = [(plan_full+exe0,exe1)]
yrange = (1,1)
plt.broken_barh(xranges, yrange, facecolors='blue')
xranges = [(plan_full+exe0+exe1,exe2)]
yrange = (0,1)
plt.broken_barh(xranges, yrange, facecolors='blue')

 

# Give the title for the broken horizontal bar chart

 

# Give x axis label for the broken horizontal bar chart

plt.xlabel('time (s)')

 

# Give y axis label for for the broken horizontal bar chart

plt.ylabel('Date')

 
plt.grid()

plt.show()

In [ ]:
plan_full

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
 
plan0T=np.mean(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000
plan1T=np.mean(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000
plan2T=np.mean(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000
# Rainfall data - 2nd day of the month for plotting broken horizontal bar chart

exe0, exe1, exe2 = np.mean(np.divide(np.array(traj_dur_list_list, dtype=np.float), traj_freq), axis=0)

yrange = (5,1)
xranges = [(0,plan0T)]
plt.broken_barh(xranges, yrange, facecolors='red')
yrange = (3,1)
xranges = [(plan0T,plan1T)]
plt.broken_barh(xranges, yrange, facecolors='red')
yrange = (1,1)
xranges = [(plan0T+exe0,plan2T)]
plt.broken_barh(xranges, yrange, facecolors='red')



xranges = [(plan0T,exe0)]
yrange = (4,1)
plt.broken_barh(xranges, yrange, facecolors='blue')

xranges = [(plan0T+exe0,exe1)]
yrange = (2,1)
plt.broken_barh(xranges, yrange, facecolors='blue')
xranges = [(plan0T+exe0+exe1,exe2)]
yrange = (0,1)
plt.broken_barh(xranges, yrange, facecolors='blue')

 

# Give the title for the broken horizontal bar chart

 

# Give x axis label for the broken horizontal bar chart

plt.xlabel('time (s)')

 

# Give y axis label for for the broken horizontal bar chart

plt.ylabel('Date')

 
plt.grid()

plt.show()

In [ ]:
plan0T

In [ ]:
print("total : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['one_plan'])[np.where(result_list)[0]])/1000))

print("plan0 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan0'])[np.where(result_list)[0]])/1000))
print("plan1 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan1'])[np.where(result_list)[0]])/1000))
print("plan2 : {} / {} / {} / {}".format(
                                        np.min(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.mean(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.median(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000,
                                        np.max(np.array(gtimer.timelist_dict['plan2'])[np.where(result_list)[0]])/1000))